This is a standalone notebook to solve a jigsaw puzzle

# Import dependencies

In [ ]:
%pip install matplotlib opencv-python scipy ipyplot;

In [ ]:
import matplotlib.pyplot as plt
import ipyplot
import numpy as np
import cv2
import math
import statistics
from scipy.signal import find_peaks
from collections import Counter

# Add utilities

In [ ]:
def imshow(img):
    plt.imshow(img[0:1680, 0:1700])

class Item():
    def __init__(self, **kwargs):
        self.update(**kwargs)

    def update(self, **kwargs):
        self.__dict__.update(kwargs)

class LoopingList(list):
    def __getitem__(self, i):
        if isinstance(i, int):
            return super().__getitem__(i % len(self))
        else:
            return super().__getitem__(i)

# Detect pieces

In [ ]:
img_rgb = cv2.imread("jigsawsqr.png")
h, w = img_rgb.shape[:2]
img_rgb = cv2.resize(img_rgb, (4*w, 4*h))
imshow(img_rgb);

In [ ]:
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
imshow(img_gray);

In [ ]:
_, img_binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
contours = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0]
img = img_rgb.copy()
cv2.drawContours(img, contours, -1, (0, 255, 0), 4)
print("Number of detected contours: ", len(contours))
imshow(img);

In [ ]:
regions = [Item(contour=contour, area=cv2.contourArea(contour), rect=cv2.boundingRect(contour)) for contour in contours]
median_area = statistics.median([region.area for region in regions])
min_area = 0.5 * median_area
max_area = 2 * median_area
print(f"Ignore too big or small regions, median_area: {median_area}")

regions = [region for region in regions if 0.5 < region.area / median_area < 2]
print(f"Number of remaining regions: {len(regions)}")

img = img_rgb.copy()
for region in regions:
    x, y, w, h = region.rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)

imshow(img);

In [ ]:
imgs = []
pieces = []
for region in regions:
    x, y, w, h = region.rect
    p = np.array([x, y])
    col = int((x - w/2) * 13 / 4540)
    row = int(1 + (y - h/2) * 13 / 4450)
    name = chr(ord('A') + col) + str(row)
    piece = Item(
        contour=region.contour - p,
        area=region.area,
        img_rgb=img_rgb[y:y+h, x:x+w],
        img_gray=img_gray[y:y+h, x:x+w],
        size=np.array([h, w]),
        name=name
    )
    pieces.append(piece)

    img = piece.img_rgb.copy()
    cv2.drawContours(img, piece.contour, -1, (0, 255, 0), 3)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=75, max_images=10);

In [ ]:
ipyplot.plot_images([piece.img_rgb for piece in pieces], labels=[piece.name for piece in pieces], img_width=75, max_images=10)

# Analyze pieces

## Detect corners

In [ ]:
imgs = []
for piece in pieces:
    min_area_rect = cv2.minAreaRect(piece.contour)
    (cx, cy), (sx, sy), angle_degrees = min_area_rect
    if sy < sx:
        angle_degrees = (angle_degrees + 90) % 360
        sx, sy = sy, sx
    min_area_rect = ((cx, cy), (sx, sy), angle_degrees)
    piece.update(
        min_area_rect=min_area_rect,
        angle_degrees=angle_degrees
    )

    img = piece.img_rgb.copy()
    box = np.int0(cv2.boxPoints(min_area_rect))
    cv2.drawContours(img, [box], 0, (0, 255, 0), 3)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=75, max_images=10);

In [ ]:
for piece in pieces:
    h, w = piece.size
    img = cv2.linearPolar(piece.img_gray, (w/2, h/2), max(h, w), cv2.WARP_FILL_OUTLIERS)
    y0 = int(piece.angle_degrees * h / 360) % 360
    img = np.concatenate([img[y0:, :], img[:y0, :]])
    _, img_binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    piece.update(img_polar=img_binary)

ipyplot.plot_images([piece.img_polar for piece in pieces], img_width=75, max_images=10); 

In [ ]:
imgs = []

for piece in pieces:
    img_reversed = np.flip(piece.img_polar, axis=1)
    h, w = piece.size
    radius = w - np.argmax(img_reversed, axis=1)
    peak_indices = find_peaks(radius, prominence=5)[0]

    polar_peaks = [(radius[idx] * max(h, w) / w, idx * 360 / h) for idx in peak_indices]
    piece.update(polar_peaks=polar_peaks)   # list of [(peak_radius, peak_degrees)]

    img = piece.img_polar.copy()
    for idx in peak_indices:
        cv2.circle(img, (radius[idx], idx), 15, 180, 2)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=150, max_images=10);

In [ ]:
imgs = []

for piece in pieces:
    h, w = piece.size

    img = piece.img_rgb.copy()
    for peak_radius, peak_degrees in piece.polar_peaks:
        peak_radians = math.radians(peak_degrees + piece.angle_degrees)
        x = w/2 + peak_radius * math.cos(peak_radians)
        y = h/2 + peak_radius * math.sin(peak_radians)
        cv2.circle(img, (int(x), int(y)), 15, (0, 200, 0), 2)

    peaks = {}  # key=quarter, value=[(abs(angle), angle, distance)] where angles are relative to the piece_angle
    for peak_radius, peak_degrees in piece.polar_peaks:
        quarter = peak_degrees // 90
        diff_degrees = peak_degrees % 180
        if diff_degrees > 90:
            diff_degrees -= 180
        peaks.setdefault(quarter, []).append((abs(diff_degrees), peak_radius, peak_degrees + piece.angle_degrees))

    corners = LoopingList()
    for quarter in range(4):
        _, peak_radius, peak_degrees = min(peaks.get(3 - quarter, [(0, 0, 0)]))
        peak_radians = math.radians(peak_degrees)
        x = w/2 + peak_radius * math.cos(peak_radians)
        y = h/2 + peak_radius * math.sin(peak_radians)
        corner = Item(point=np.array([int(x), int(y)]))
        corners.append(corner)

    for idx, corner in enumerate(corners):
        corner.update(prev=corners[idx-1], next=corners[idx+1])
    piece.update(corners=corners)
    
    cv2.line(img, piece.corners[0].point, piece.corners[2].point, (255, 0, 0), 3)
    cv2.line(img, piece.corners[1].point, piece.corners[3].point, (255, 0, 0), 3)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=75, max_images=10);

## Analyze edges

In [ ]:
imgs = []

def length(v):
    return v[0]**2 + v[1]**2

def sub_contour(c, idx0, idx1):
    if idx1 > idx0:
        return c[idx0:idx1]
    else:
        return np.concatenate([c[idx0:], c[:idx1]])

for piece in pieces:
    for corner in piece.corners:
        distances = [length(p[0] - corner.point) for p in piece.contour]
        corner.update(idx=distances.index(min(distances)))
    
    edges = LoopingList()
    for corner in piece.corners:
        corner0 = corner.prev
        corner1 = corner
        contour = sub_contour(piece.contour, corner0.idx, corner1.idx)
        edge = Item(
            contour=contour,
            corner0=corner0,
            corner1=corner1,
            p0=corner0.point,
            p1=corner1.point,
            idx0=corner0.idx,
            idx1=corner1.idx
        )
        edges.append(edge)
    
    piece.update(edges=edges)

    for edge in piece.edges:
        edge.corner0.update(edge1=edge)
        edge.corner1.update(edge0=edge)

    img = piece.img_rgb.copy()
    cv2.drawContours(img, piece.edges[0].contour, -1, (0, 255, 0), 6)
    cv2.drawContours(img, piece.edges[1].contour, -1, (255, 0, 0), 6)
    cv2.drawContours(img, piece.edges[2].contour, -1, (0, 255, 0), 6)
    cv2.drawContours(img, piece.edges[3].contour, -1, (255, 0, 0), 6)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=75, max_images=10);

In [ ]:
imgs = []
NB_SAMPLES = 100  # size of the sampled contour

for piece in pieces:
    nb_flats = 0
    img = piece.img_rgb.copy()
    for idx, edge in enumerate(piece.edges):
        dx, dy = edge.p1 - edge.p0
        angle_radians = math.atan2(dy, dx)
        sin = math.sin(angle_radians)
        cos = math.cos(angle_radians)
        matrix = np.array([[cos, -sin], [sin, cos]])
        normalized_contour = (edge.contour - edge.p0) @ matrix  # first point at (0, 0), last point at (X, 0)
        sample_idx = [i * (len(normalized_contour)-1) // (NB_SAMPLES-1) for i in range(NB_SAMPLES)]
        sampled_contour = normalized_contour.astype(np.int64)[sample_idx, 0, :]
        heights = normalized_contour[:,0,1]
        min_height = min(heights)
        max_height = max(heights)
        if abs(max_height) + abs(min_height) < 10:
            edge_type = 0
            nb_flats += 1
            color = (0, 255, 0)
        elif abs(max_height) > abs(min_height):
            edge_type = 1
            color = (255, 0, 0)
        else:
            edge_type = -1
            color = (0, 0, 255)
        edge.update(
            idx=idx,
            arc_length=cv2.arcLength(edge.contour, closed=False),
            straight_length=math.sqrt(dx**2 + dy**2),
            height=max(abs(min_height), abs(max_height)),
            angle_degrees=math.degrees(angle_radians),
            type=edge_type,
            prev=piece.edges[idx-1],
            next=piece.edges[idx+1]
        )
        edge.update(
            normalized_prev_point=(edge.prev.p0 - edge.p0) @ matrix,
            normalized_next_point=(edge.next.p1 - edge.p1) @ matrix,
            normalized_contour=normalized_contour.astype(np.int64),
            sampled_contour=sampled_contour
        )
        cv2.drawContours(img, edge.contour, -1, color, 6)

    piece.update(nb_flats=nb_flats)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=75, max_images=10);

In [ ]:
piece = pieces[0]
edge = piece.edges[0]
plt.plot(edge.normalized_contour[:, :, 0], edge.normalized_contour[:, :, 1])
plt.plot(edge.sampled_contour[:, 0], edge.sampled_contour[:, 1])

In [ ]:
ipyplot.plot_class_tabs(images=[piece.img_rgb for piece in pieces], labels=[piece.nb_flats for piece in pieces], img_width=50);

In [ ]:
imgs = []
PAD = 50

for idx, piece in enumerate(pieces):
    h, w = piece.size
    for edge in piece.edges:
        img = cv2.copyMakeBorder(piece.img_rgb, PAD, PAD, PAD, PAD, cv2.BORDER_CONSTANT)
        matrix = cv2.getRotationMatrix2D((PAD + w/2, PAD + h/2), edge.angle_degrees, 1)
        cv2.warpAffine(img, matrix, (w+2*PAD, h+2*PAD), img)
        contour = cv2.transform(edge.contour + PAD, matrix)
        p0 = contour[0][0]
        p1 = contour[-1][0]
        p_prev = p0 + edge.normalized_prev_point
        p_next = p1 + edge.normalized_next_point
        cv2.circle(img, (int(p0[0]), int(p0[1])), 15, (0, 255, 0), 3)
        cv2.circle(img, (int(p_prev[0]), int(p_prev[1])), 25, (0, 255, 0), 3)
        cv2.circle(img, (int(p1[0]), int(p1[1])), 15, (255, 0, 0), 3)
        cv2.circle(img, (int(p_next[0]), int(p_next[1])), 25, (255, 0, 0), 3)
        cv2.line(img, (0, int(p0[1])), (w + 2*PAD, int(p0[1])), (0, 255, 0), 2)
        imgs.append(img)

ipyplot.plot_images(imgs, img_width=75);

## Add piece utilities

In [ ]:
piece_by_name = dict([(piece.name, piece) for piece in pieces])

def transform_idx(piece, contour_idx, transform):
    return cv2.transform(piece.contour[contour_idx:contour_idx+1], transform)[0][0]

def make_transform(piece, contour_idx, target_position, angle_degrees):
    """ Compute the affine transform of the piece that rotates by angle_degrees
    and set the point piece.contour[idx] at position"""
    rotation_matrix = cv2.getRotationMatrix2D((0, 0), angle_degrees, 1)
    position = transform_idx(piece, contour_idx, rotation_matrix)
    dx, dy = target_position - position
    rotation_matrix33 = np.concatenate([rotation_matrix, [[0, 0, 1]]])
    translation_matrix = np.array([[1, 0, dx], [0, 1, dy], [0, 0, 1]])
    transform = translation_matrix @ rotation_matrix33
    return transform[:2]

def draw_piece_on_edge(piece, edge):
    PAD = 50
    h, w = piece.size
    img = cv2.copyMakeBorder(piece.img_rgb, PAD, PAD, PAD, PAD, cv2.BORDER_CONSTANT)
    matrix = cv2.getRotationMatrix2D((PAD + w/2, PAD + h/2), edge.angle_degrees, 1)
    cv2.warpAffine(img, matrix, (w+2*PAD, h+2*PAD), img)
    return img

def first_flat_edge(piece):
    return [edge for edge in piece.edges if edge.type == 0 and edge.prev.type != 0][0]

def last_flat_edge(piece):
    return [edge for edge in piece.edges if edge.type == 0 and edge.next.type != 0][0]

def edge_after_flat(piece):
    return last_flat_edge(piece).next

def edge_before_flat(piece):
    return first_flat_edge(piece).prev

# Compute puzzle size

In [ ]:
def compute_size(area, perimeter):
    # perimeter = 2 * (H+W)
    # area = H*W
    # H**2 - perimeter/2 * H + area = 0
    a = 1
    b = -perimeter/2
    c = area
    delta = b**2 - 4*a*c
    h = int((-b - math.sqrt(delta)) / (2*a))
    w = int((-b + math.sqrt(delta)) / (2*a))
    return (min(h, w), max(h, w))

In [ ]:
solution = Item()

nb_flats = Counter([piece.nb_flats for piece in pieces])
assert nb_flats[2] == 4
area = len(pieces)
perimeter = nb_flats[1] + 2*nb_flats[2]
w, h = compute_size(area, perimeter)
print(f"Size of puzzle grid: {w} x {h}")
assert w * h == area
assert 2 * (w + h) == perimeter

solution.update(grid_size = (w, h))

In [ ]:
area = sum([piece.area for piece in pieces])
perimeter = sum([edge.straight_length for piece in pieces for edge in piece.edges if edge.type == 0])

w, h = compute_size(area, perimeter)
print(f"Approximate size of puzzle image: {w} x {h}")

solution.update(approximate_size = (w, h))

# Compute the border

## Add border utilities

In [ ]:
border_pieces = [piece for piece in pieces if piece.nb_flats > 0]

def hardcoded_piece_after_flat(piece):
    sequence = LoopingList([
        'A1', 'G2', 'D3', 'L4', 'E4', 'F3', 'E3', 'B11', 'H1', 'G12', 'E10',
        'H4', 'G6', 'D9', 'F2', 'H6', 'H12', 'I6', 'E7', 'K2', 'L9', 'K11',
        'A12', 'I7', 'C11', 'A2', 'H3', 'D10', 'I5', 'D5', 'E12', 'G9', 'B5',
        'F12', 'H11', 'A11', 'L10', 'D4', 'L6', 'K3', 'F4', 'A10', 'A5', 'B2'])
    idx = sequence.index(piece.name)
    return piece_by_name[sequence[idx+1]]

def evaluate_border_matcher(matcher):
    results = [(matcher(piece).index(hardcoded_piece_after_flat(piece)), piece.name, piece) for piece in border_pieces]
    results.sort()
    errors = sum([result[0] for result in results])
    worst = max(results)
    print(f"border matcher errors: {errors}, worst: {worst[:2]}")
    return errors, worst

def draw_border_candidates(piece0, matcher):
    imgs = [draw_piece_on_edge(piece0, last_flat_edge(piece0))]
    print(f"0 {piece0.name}")
    for idx, piece1 in enumerate(matcher(piece0)[:10]):
        print(f"{idx+1} {piece1.name}")
        imgs.append(draw_piece_on_edge(piece1, first_flat_edge(piece1)))
    return imgs

## Feature match

In [ ]:
before_flat_features = {}  # key=piece, value=features
after_flat_features = {}  # key=piece, value=features
for piece in border_pieces:
    edge = first_flat_edge(piece)
    features = np.array([
        edge.prev.type,
        edge.prev.straight_length,
        edge.prev.arc_length,
        edge.prev.height,
        edge.normalized_prev_point[0],
        edge.normalized_prev_point[1]
    ])
    before_flat_features[piece] = features

    edge = last_flat_edge(piece)
    features = np.array([
        edge.next.type,
        edge.next.straight_length,
        edge.next.arc_length,
        edge.next.height,
        edge.normalized_next_point[0],
        edge.normalized_next_point[1]
    ])
    after_flat_features[piece] = features

In [ ]:
male_after_flat_features = np.array([f for f in after_flat_features.values() if f[0] == 1])
female_before_flat_features = np.array([f for f in before_flat_features.values() if f[0] == -1])
male_before_flat_features = np.array([f for f in before_flat_features.values() if f[0] == 1])
female_after_flat_features = np.array([f for f in after_flat_features.values() if f[0] == -1])

index_x = 1
index_y = 2
index_c = 3
f = male_before_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker=".")
f = female_after_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker="*");
f = male_after_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker="v")
f = female_before_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker="^")

In [ ]:
index_x = 4
index_y = 5
index_c = 1
f = male_before_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker=".")
f = female_after_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker="*")
f = male_after_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker="v")
f = female_before_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker="^");

In [ ]:
def feature_matches_after_flat(piece0):
    features0 = after_flat_features[piece0]
    results = [(sum((features1[1:] - features0[1:])**2), piece1) for piece1, features1 in before_flat_features.items() if features0[0] == -features1[0]]
    results.sort()
    return [piece for score, piece in results]

errors, worst = evaluate_border_matcher(feature_matches_after_flat)
imgs = draw_border_candidates(worst[2], feature_matches_after_flat)
ipyplot.plot_images(imgs, img_width=75);

## Xor border match

In [ ]:
imgs = []

piece0 = border_pieces[0]
edge0 = last_flat_edge(piece0)

piece1 = feature_matches_after_flat(piece0)[0]
edge1 = last_flat_edge(piece1)

imgs.append(draw_piece_on_edge(piece0, edge0))
imgs.append(draw_piece_on_edge(piece1, edge1))

PAD = 50
size = max(piece0.img_gray.shape[:2]) + 2*PAD
img0 = np.zeros((size, size), np.uint8)
ref_point = np.array([size // 2, size - PAD])
transform0 = make_transform(piece0, edge0.idx1, ref_point, edge0.angle_degrees)
contour0 = cv2.transform(piece0.contour, transform0)
cv2.fillPoly(img0, [contour0], 255)
imgs.append(img0)

transform1 = make_transform(piece1, edge1.idx0, ref_point, edge1.angle_degrees)
contour1 = cv2.transform(piece1.contour, transform1)
img1 = np.zeros_like(img0)
cv2.fillPoly(img1, [contour1], 255)
imgs.append(img1)

img_both = np.zeros_like(img0)
cv2.drawContours(img_both, [contour0, contour1], -1, 255, 2)
imgs.append(img_both)

img_mask = np.zeros_like(img0)
cv2.polylines(img_mask, [sub_contour(contour0, edge0.next.idx0 + 10, edge0.next.idx1 - 10)], False, 255, 20)
imgs.append(img_mask)

img0_edge = np.zeros_like(img0)
cv2.bitwise_and(img0, img_mask, img0_edge)
imgs.append(img0_edge)

img1_edge = np.zeros_like(img0)
cv2.bitwise_and(img1, img_mask, img1_edge)
imgs.append(img1_edge)

img_xor = np.zeros_like(img0)
cv2.bitwise_xor(img0_edge, img1_edge, img_xor)
imgs.append(img_xor)

img_and = np.zeros_like(img0)
cv2.bitwise_and(img0_edge, img1_edge, img_and)
imgs.append(img_and)

img_or = np.zeros_like(img0)
cv2.bitwise_or(img0_edge, img1_edge, img_or)
imgs.append(img_or)

print("Xor match:", img_xor.shape[:2], np.sum(img_xor), np.sum(img_mask), np.sum(img_xor) / np.sum(img_mask))
ipyplot.plot_images(imgs, img_width=100);

## Cv border match

In [ ]:
def cv_matches_after_flat(piece0):
    edge0 = last_flat_edge(piece0)
    results = []
    for piece1 in border_pieces:
        edge1 = first_flat_edge(piece1)
        if edge1.prev.type == -edge0.next.type:
            score = cv2.matchShapes(edge1.prev.contour, edge0.next.contour, 1, 0.)
            result = (score, piece1)
            results.append(result)
    results.sort()
    return [piece for score, piece in results]

errors, worst = evaluate_border_matcher(cv_matches_after_flat)
imgs = draw_border_candidates(worst[2], cv_matches_after_flat)
ipyplot.plot_images(imgs, img_width=75);

## Distance border match

In [ ]:
imgs = []

piece0 = border_pieces[1]
edge0 = last_flat_edge(piece0)
contour0 = edge0.next.sampled_contour

piece1 = feature_matches_after_flat(piece0)[0]
edge1 = first_flat_edge(piece1)
contour1 = edge1.prev.sampled_contour

diff = contour0[::-1] + contour1
offset = np.mean(diff, axis=0)
plt.plot(contour0[:, 0], contour0[:, 1])
plt.plot(offset[0]-contour1[:, 0], offset[1]-contour1[:, 1])
print(np.sum((diff - offset)**2))

In [ ]:
def distance_matches_after_flat(piece0):
    edge0 = edge_after_flat(piece0)
    contour0 = edge0.sampled_contour[::-1]

    results = []
    for piece1 in pieces:
        if piece1.nb_flats > 0:
            edge1 = edge_before_flat(piece1)
            if edge1.type == -edge0.type:
                contour1 = edge1.sampled_contour
                diff = contour0 + contour1
                offset = np.mean(diff, axis=0)
                score = np.sum((diff - offset)**2)
                results.append((score, piece1))
    results.sort()
    return [piece for score, piece in results]

errors, worst = evaluate_border_matcher(distance_matches_after_flat)
imgs = draw_border_candidates(worst[2], distance_matches_after_flat)
ipyplot.plot_images(imgs, img_width=75);

# Place the border

In [ ]:
PAD = 30
size = max(solution.approximate_size) + 2*PAD

solution.update(
    img_rgb=np.zeros((size, size, 3), img_rgb.dtype),
    grid={} # key=(i, j), value=(piece, top_edge_idx)
)

def place_piece(pos, piece, top_edge_idx):
    x, y = pos
    c = int((x+y)%2 * 255)
    solution.grid[pos] = (piece, top_edge_idx)
    contour = cv2.transform(piece.contour, piece.transform)
    cv2.drawContours(solution.img_rgb, contour, -1, (c, 255 - c, 255), 5)

def place_top_left():
    piece = piece_by_name['A1']
    top_edge = first_flat_edge(piece)
    corner = top_edge.corner1
    transform = make_transform(piece, corner.idx, np.array([PAD, PAD]), top_edge.angle_degrees + 180)
    piece.update(transform=transform)
    place_piece((0, 0), piece, top_edge.idx)

def place_border(pos, dpos, quarter):
    pos = np.array(pos)
    dpos = np.array(dpos)
    for _ in range(11):
        piece, top_edge_idx = solution.grid[tuple(pos)]
        pos += dpos
        if tuple(pos) in solution.grid:
            break
        flat_edge = piece.edges[top_edge_idx - quarter]
        ref_point = transform_idx(piece, flat_edge.corner1.idx, piece.transform)
        piece = feature_matches_after_flat(piece)[0]
        flat_edge = first_flat_edge(piece)
        transform = make_transform(piece, flat_edge.idx0, ref_point, flat_edge.angle_degrees + 180 - 90 * quarter)
        piece.update(transform=transform)
        place_piece(tuple(pos), piece, flat_edge.idx + quarter)

place_top_left()
place_border((0, 0), (0, 1), 3)
place_border((0, 11), (1, 0), 2)
place_border((11, 11), (0, -1), 1)
place_border((11, 0), (-1, 0), 0)

plt.imshow(solution.img_rgb)
plt.axis("off");